# Neural network regressor with Keras

Created on JupyterLab

## Introduction


Keras is a high-level API for building deep learning models. It has gained favor for its ease of use and syntactic simplicity facilitating fast development. Building a very complex deep learning network can be achieved with Keras with only few lines of code. 

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>
    
1. <a href="#item31">Download and Clean Dataset</a>  
2. <a href="#item32">Import Keras</a>  
3. <a href="#item33">Build a Neural Network</a>  
4. <a href="#item34">Train and Test the Network</a>  

</font>
</div>


<a id="item31"></a>


## Download and Clean Dataset


Let's start by importing the <em>pandas</em> and the Numpy libraries.


In [1]:
#!pip install numpy==1.21.4
#!pip install pandas==1.3.4
#!pip install keras==2.1.6

In [2]:
import pandas as pd
import numpy as np

We will be playing around with the same dataset that we used in the videos.

The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:</strong>

1. Cement

2. Blast Furnace Slag

3. Fly Ash

4. Water

5. Superplasticizer

6. Coarse Aggregate

7. Fine Aggregate

Let's download the data and read it into a <em>pandas</em> dataframe.


In [3]:
concrete_data = pd.read_csv('concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


So the first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa. 


In [4]:
concrete_data.shape

(1030, 9)

So, there are approximately 1000 samples to train our model on. Because of the few samples, we have to be careful not to overfit the training data.


Let's summarise and check the dataset for any missing values.

In [5]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [6]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.


#### Split data into predictors and target


**The target variable in this problem is the concrete sample strength**. Therefore, our predictors will be all the other columns.

In [7]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

Let's do a quick sanity check of the predictors and the target dataframes.


In [8]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [9]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Finally, the last step is to normalize the data by substracting the mean and dividing by the standard deviation.


In [10]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


Let's save the number of predictors to *n_cols* since we will need this number when building our network.


In [11]:
n_cols = predictors_norm.shape[1] # number of predictors

## Import Keras


Keras normally runs on top of a low-level library such as TensorFlow. This means that to be able to use the Keras library, you will have to install TensorFlow first.

#### Let's go ahead and import the Keras library


In [12]:
import keras

Let's import the rest of the packages from the Keras library that we will need to build our regression model.

In [13]:
from keras.models import Sequential
from keras.layers import Dense

## Build a Neural Network


Let's define a function that defines our regression model for us so that we can conveniently call it to create our model.


In [14]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(50, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

The above function create a model that has two hidden layers, each of 50 hidden units. The activation function is Relu. The loss function is the mean squared error (MSE).

*Increasing the number of hidden layers from single- to multiple-layer neural network usually improves model's performance by overcoming the limitation of linear modelization.*

## Train and Test the Network


Let's call the function now to create our model.


In [15]:
# build the model
model = regression_model()

**Next, we will train and test the model at the same time using the *fit* method. We will leave out 30% of the data for validation and we will train the model for 100 epochs (updating the weights and biases for a predefined number of iterations or epochs).**

In [16]:
# fit the model
# model.fit(predictors_norm, target, validation_split=0.3, epochs=100, verbose=2)

You can also randomly split the data for training and testing by holding 30% of the data for validation using scikit-learn

In [17]:
import sklearn
from sklearn.model_selection import train_test_split

In [18]:
predictors_train, predictors_test, target_train, target_test = train_test_split(
...     predictors_norm, target, test_size=0.30, random_state=42)

print ('Size for predictors training: ', predictors_train.shape)
print ('Size for predictors testing: ', predictors_test.shape)

print ('Size for target training: ', target_train.shape)
print ('Size for target testing: ', target_test.shape)

Size for predictors training:  (721, 8)
Size for predictors testing:  (309, 8)
Size for target training:  (721,)
Size for target testing:  (309,)


*Increasing number of epochs usually improves model's performance by increasing the number of iterations to update weights and biases and reduce the error.*

In [19]:
# fit the model
model.fit(predictors_train, target_train, validation_data=(predictors_test, target_test), epochs=50, verbose=2)

Epoch 1/50
23/23 - 1s - loss: 1510.7063 - val_loss: 1355.3246 - 639ms/epoch - 28ms/step
Epoch 2/50
23/23 - 0s - loss: 1349.3916 - val_loss: 1165.4166 - 73ms/epoch - 3ms/step
Epoch 3/50
23/23 - 0s - loss: 1094.8788 - val_loss: 883.2722 - 72ms/epoch - 3ms/step
Epoch 4/50
23/23 - 0s - loss: 768.7668 - val_loss: 569.2289 - 71ms/epoch - 3ms/step
Epoch 5/50
23/23 - 0s - loss: 461.4597 - val_loss: 336.9366 - 71ms/epoch - 3ms/step
Epoch 6/50
23/23 - 0s - loss: 285.9695 - val_loss: 235.9975 - 73ms/epoch - 3ms/step
Epoch 7/50
23/23 - 0s - loss: 225.1735 - val_loss: 206.6277 - 76ms/epoch - 3ms/step
Epoch 8/50
23/23 - 0s - loss: 204.2502 - val_loss: 191.5900 - 81ms/epoch - 4ms/step
Epoch 9/50
23/23 - 0s - loss: 193.1861 - val_loss: 182.0646 - 80ms/epoch - 3ms/step
Epoch 10/50
23/23 - 0s - loss: 184.5523 - val_loss: 176.5830 - 81ms/epoch - 4ms/step
Epoch 11/50
23/23 - 0s - loss: 178.1394 - val_loss: 171.2660 - 71ms/epoch - 3ms/step
Epoch 12/50
23/23 - 0s - loss: 172.5016 - val_loss: 166.9159 - 72ms

Make predictions

In [20]:
prediction = model.predict(predictors_test)
prediction.shape

10/10 [==============================] - 0s 1ms/step


(309, 1)

Evaluate the model by calculating the mean squared error

In [21]:
from sklearn.metrics import mean_squared_error
mean_squared_error(target_test, prediction)

95.74033730069718

We can repeat this process 50 times in a loop 

In [22]:
MSE = []
for x in range(1, 51):
    predictors_train = []; predictors_test=[]; target_train=[]; target_test=[]
    predictors_train, predictors_test, target_train, target_test = train_test_split(predictors_norm, target, test_size=0.30)
    # fit the model
    model.fit(predictors_train, target_train, validation_data=(predictors_test, target_test), epochs=50, verbose=2)
    prediction = []; prediction = model.predict(predictors_test)
    #prediction.shape
    mse01 = [];mse01 = mean_squared_error(target_test, prediction)
    MSE.append(mse01)    

Epoch 1/50
23/23 - 0s - loss: 87.9066 - val_loss: 91.4229 - 144ms/epoch - 6ms/step
Epoch 2/50
23/23 - 0s - loss: 83.7137 - val_loss: 84.5473 - 68ms/epoch - 3ms/step
Epoch 3/50
23/23 - 0s - loss: 80.2283 - val_loss: 86.3437 - 68ms/epoch - 3ms/step
Epoch 4/50
23/23 - 0s - loss: 76.8678 - val_loss: 80.8753 - 67ms/epoch - 3ms/step
Epoch 5/50
23/23 - 0s - loss: 74.0635 - val_loss: 77.3864 - 70ms/epoch - 3ms/step
Epoch 6/50
23/23 - 0s - loss: 70.9844 - val_loss: 73.9386 - 72ms/epoch - 3ms/step
Epoch 7/50
23/23 - 0s - loss: 68.6041 - val_loss: 72.9389 - 69ms/epoch - 3ms/step
Epoch 8/50
23/23 - 0s - loss: 65.7991 - val_loss: 72.0548 - 66ms/epoch - 3ms/step
Epoch 9/50
23/23 - 0s - loss: 63.7078 - val_loss: 67.0154 - 70ms/epoch - 3ms/step
Epoch 10/50
23/23 - 0s - loss: 60.9735 - val_loss: 65.6379 - 72ms/epoch - 3ms/step
Epoch 11/50
23/23 - 0s - loss: 59.0107 - val_loss: 63.5619 - 158ms/epoch - 7ms/step
Epoch 12/50
23/23 - 0s - loss: 57.0769 - val_loss: 61.5455 - 72ms/epoch - 3ms/step
Epoch 13/50

In [23]:
mean_MSE = np.mean(MSE)
std_MSE = np.std(MSE)
RMSE = np.sqrt(mean_MSE)
print("Mean of Mean Squares Errors (normalized data) =: %.2f" % mean_MSE)
print("Standard Deviation of Mean Squares Errors (normalized data) =: %.2f" % std_MSE)
print("Square root of MSE: Root Mean Squared Error (normalized data) =: %.2f" % RMSE)

Mean of Mean Squares Errors (normalized data) =: 11.90
Standard Deviation of Mean Squares Errors (normalized data) =: 6.69
Square root of MSE: Root Mean Squared Error (normalized data) =: 3.45


# Authors' rights

Machine Learning with Python course by IBM on Coursera: https://www.coursera.org/learn/machine-learning-with-python/

Completed and modified by Mathilde Marie Duville as part of the IBM Artificial Intelligence Engineering Professional Certificate and corresponding IBM badges. Please, follow the subsequent links to confirm the accreditation:

https://www.coursera.org/account/accomplishments/professional-cert/KSLW773DAATP?utm_source=link&utm_medium=certificate&utm_content=cert_image&utm_campaign=sharing_cta&utm_product=prof

https://www.credly.com/users/mathilde-marie-duville/badges

-------------------------------------------------------------------------------------------------------------------------------------------

Author: [Alex Aklson](https://www.linkedin.com/in/aklson/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDL0101ENSkillsNetwork945-2022-01-01).

Change Log

|  Date (YYYY-MM-DD) |  Version | Changed By  |  Change Description |
|---|---|---|---|
| 2020-09-21  | 2.0  | Srishti  |  Migrated Lab to Markdown and added to course repo in GitLab |


© IBM Corporation 2020. All rights reserved.

This notebook is part of a course on **Coursera** called *Introduction to Deep Learning & Neural Networks with Keras*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0101EN_Coursera_Week3_LAB1).


<hr>

Copyright &copy; 2019 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_medium=dswb&utm_source=bducopyrightlink&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDL0101ENSkillsNetwork945-2022-01-01&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDL0101ENSkillsNetwork945-2022-01-01).
